In [1]:
from sentence_transformers import SentenceTransformer, util
import re, emoji, pythainlp, unicodedata
import pandas as pd

In [2]:
model = SentenceTransformer("./paraphrase-multilingual-mpnet-base-v2/")

sent1 = model.encode("ไก่ลุงเคนกินแล้วมีแต่สารเร่งโต", convert_to_tensor=True)

In [3]:
sent1

tensor([ 3.2768e-02,  6.8993e-02, -1.4386e-02,  4.7347e-03,  6.1043e-02,
         7.0178e-02, -1.6894e-01,  9.5604e-02,  6.3860e-04, -6.2015e-03,
        -4.0120e-02,  1.1180e-01,  5.5575e-02,  3.7014e-02,  8.8635e-02,
        -1.6915e-02, -3.2925e-02,  1.3172e-01, -5.2193e-02,  1.3172e-02,
        -4.6006e-02, -4.8764e-02,  1.0579e-01,  2.8331e-02, -8.0242e-02,
        -3.3475e-02,  3.1135e-02,  6.5706e-02,  4.0724e-02, -9.2531e-02,
         4.3366e-02, -6.9590e-02, -6.0008e-04, -5.4086e-03,  5.7600e-02,
         7.7187e-02,  3.0827e-02,  8.7980e-02, -1.2877e-01,  2.0610e-01,
         2.1590e-02,  2.4679e-02, -6.8451e-02,  4.6766e-02, -5.3660e-03,
        -1.4003e-01,  4.0191e-02, -4.8288e-02, -2.5077e-02, -4.0389e-03,
         4.8187e-02, -4.4681e-02, -3.2831e-02,  5.8660e-02,  1.4492e-01,
         6.4855e-04, -9.2286e-02, -4.3360e-02,  1.2411e-02, -8.7624e-02,
        -7.2196e-03, -4.7196e-02,  5.7665e-02,  2.4946e-02, -9.9473e-02,
         3.8929e-02,  7.2554e-02,  1.4231e-03,  1.5

In [17]:
def normalize_double_quote(text: str):
    all_quote = [
    '“', # U+201c
    '”', # U+201d
    "„", # U+201e
    '«', # U+00AB
    '»', # U+00BB
    '„', # U+201E
    '“', # U+201C
    '‟', # U+201F
    '”', # U+201D
    '❝', # U+275D
    '❞', # U+275E
    '〝', # U+301D
    '〞', # U+301E
    '〟', # U+301F
    '＂', # U+FF02
    ]
    std_quote = "\"" #U+0022
    table = str.maketrans(dict.fromkeys(all_quote, std_quote))
    return text.translate(table)

def normalize_single_quote(text: str):
    all_quote = ['\u02BB', '\u02BC', '\u066C', '\u2018', '\u2019', '\u201A', '\u275B', '\u275C']
    std_quote = "\'"
    table = str.maketrans(dict.fromkeys(all_quote, std_quote))
    return text.translate(table)

def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}
]+|
[^\s()]*?\)|
)+(?:
[^\s()]*?\)|
|[^\s`!()
{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_rep(text):
    def _replace_rep(m):
        c,cc = m.groups()
        return f'{c}xxrep'
    re_rep = re.compile(r'(\S)(\1{2,})')
    return re_rep.sub(_replace_rep, text)

def ungroup_emoji(toks):
    res = []
    for tok in toks:
        if emoji.emoji_count(tok) == len(tok):
            for char in tok:
                res.append(char)
        else:
            res.append(tok)
    return res

def preprocess_func(text):
    new_text = unicodedata.normalize('NFKD', text)
    new_text = pythainlp.util.normalize(new_text)
    new_text = normalize_double_quote(new_text)
    new_text = normalize_single_quote(new_text)

    new_text = new_text.lower().strip()
    new_text = replace_url(new_text)
    new_text = replace_rep(new_text)

    res = [word for word in pythainlp.word_tokenize(new_text) if word and not re.search(pattern=r"\s+", string=word) and word not in pythainlp.corpus.thai_stopwords()]

    res = ungroup_emoji(res)
    
    res = ' '.join(res)
    return res

In [18]:
train_df = pd.read_csv('./train_data_fake_news.csv')
val_df = pd.read_csv('./valid_data_fake_news.csv')
test_df = pd.read_csv('./test_data_fake_news.csv')

In [19]:
train_df['sent'] = train_df['text'].map(preprocess_func)

In [21]:
train_df

,text,label,sent
0,ไก่ลุงเคนกินแล้วมีแต่สารเร่งโต,1,ไก่ ลุง เคน กิน สาร เร่ง
1,เครื่องดื่มต้านCovid-19ค่าพีเอชคือความเป็นด่าง...,1,เครื่องดื่ม ต้าน covid- 19 ค่า พี เอช ด่าง กระ...
2,"แค่ฟังชื่อก็ขนลุก!""ต้นป่าช้าเหงา""สมุนไพรชั้นดี...",1,"ฟัง ชื่อ ขนลุก !"" ต้น ป่าช้า เหงา "" สมุนไพร ชั..."
3,พิชัย'แนะ'ประยุทธ์'เพิ่มเซลล์สมองแก้ปัญหาเศรษฐ...,0,พิชัย ' แนะ ' ประยุทธ์ ' เซลล์ สมอง แก้ปัญหา เ...
4,อุจจาระตกค้างเรื่องใหญ่!แจกสูตรดีท็อกซ์ล้างลำไ...,1,อุจจาระ ตกค้าง เรื่องใหญ่ ! แจก สูตร ดีท็อกซ์ ...
...,...,...,...
2358,เดี๋ยวนี้มีใบทุเรียนน้ำแกโรคมะเร็งหายขาดจ้า,1,เดี๋ยวนี้ ใบ ทุเรียน น้ำ โรคมะเร็ง หายขาด
2359,อาหาร8ประเภทที่ไม่ควรนำไปอุ่นซ้ำอีกรอบ|||จะต้อ...,1,อาหาร 8 ประเภท อุ่น ซ้ำ อี กรอบ | xxrep เกริ่น...
2360,เภสัชเผยยากลุ่มNSAIDsห้ามกินร่วมกันเสี่ยงอันตร...,0,เภสัช เผย ยา nsaids ห้าม กิน เสี่ยงอันตราย ถึง...
2361,สบส.ฟันธง!!วัยทำงานเกือบครึ่งอนาคตเสี่ยงโรคเรื...,0,สบ ส. ฟันธง !! วัยทำงาน ครึ่ง อนาคต เสี่ยง โรค...


In [22]:
train_df['embeddings'] = train_df['sent'].map(model.encode)

In [23]:
train_df

,text,label,sent,embeddings
0,ไก่ลุงเคนกินแล้วมีแต่สารเร่งโต,1,ไก่ ลุง เคน กิน สาร เร่ง,"[0.024978174, 0.09388084, -0.016153483, 0.0162..."
1,เครื่องดื่มต้านCovid-19ค่าพีเอชคือความเป็นด่าง...,1,เครื่องดื่ม ต้าน covid- 19 ค่า พี เอช ด่าง กระ...,"[-0.050368443, -0.11657548, -0.011021655, 0.01..."
2,"แค่ฟังชื่อก็ขนลุก!""ต้นป่าช้าเหงา""สมุนไพรชั้นดี...",1,"ฟัง ชื่อ ขนลุก !"" ต้น ป่าช้า เหงา "" สมุนไพร ชั...","[0.088294804, 0.0630195, -0.015678342, 0.02156..."
3,พิชัย'แนะ'ประยุทธ์'เพิ่มเซลล์สมองแก้ปัญหาเศรษฐ...,0,พิชัย ' แนะ ' ประยุทธ์ ' เซลล์ สมอง แก้ปัญหา เ...,"[0.014535166, -0.0018562346, -0.011774885, 0.0..."
4,อุจจาระตกค้างเรื่องใหญ่!แจกสูตรดีท็อกซ์ล้างลำไ...,1,อุจจาระ ตกค้าง เรื่องใหญ่ ! แจก สูตร ดีท็อกซ์ ...,"[0.06875539, 0.086440444, -0.015181167, -0.050..."
...,...,...,...,...
2358,เดี๋ยวนี้มีใบทุเรียนน้ำแกโรคมะเร็งหายขาดจ้า,1,เดี๋ยวนี้ ใบ ทุเรียน น้ำ โรคมะเร็ง หายขาด,"[0.06274776, 0.14091289, -0.021309244, -0.0792..."
2359,อาหาร8ประเภทที่ไม่ควรนำไปอุ่นซ้ำอีกรอบ|||จะต้อ...,1,อาหาร 8 ประเภท อุ่น ซ้ำ อี กรอบ | xxrep เกริ่น...,"[-0.021780498, 0.044933267, -0.008214561, 0.00..."
2360,เภสัชเผยยากลุ่มNSAIDsห้ามกินร่วมกันเสี่ยงอันตร...,0,เภสัช เผย ยา nsaids ห้าม กิน เสี่ยงอันตราย ถึง...,"[-0.008500792, -0.009282524, -0.0144894505, -0..."
2361,สบส.ฟันธง!!วัยทำงานเกือบครึ่งอนาคตเสี่ยงโรคเรื...,0,สบ ส. ฟันธง !! วัยทำงาน ครึ่ง อนาคต เสี่ยง โรค...,"[-0.112570204, 0.12547264, -0.013787132, -0.14..."


In [24]:
val_df['sent'] = val_df['text'].map(preprocess_func)
val_df['embeddings'] = val_df['sent'].map(model.encode)

test_df['sent'] = test_df['text'].map(preprocess_func)
test_df['embeddings'] = test_df['sent'].map(model.encode)

In [25]:
print(len(val_df))
print(len(test_df))

506
507


In [26]:
X_train = train_df['embeddings'].to_list()
y_train = train_df['label'].to_list()

X_test = test_df['embeddings'].tolist()
y_test = test_df['label'].tolist()

In [27]:
from sklearn.linear_model import LogisticRegression

model_classification = LogisticRegression(C=2., penalty='l2', solver='liblinear', dual=False, multi_class='ovr', verbose=10)
model_classification.fit(X_train, y_train)
model_classification.score(X_test, y_test)

[LibLinear]

0.8658777120315582

In [28]:
from sklearn.metrics import classification_report
y_val_pred = model_classification.predict(X_test)
print(classification_report(y_test, y_val_pred))

              precision    recall  f1-score   support

           0       0.87      0.80      0.83       212
           1       0.86      0.91      0.89       295

    accuracy                           0.87       507
   macro avg       0.87      0.86      0.86       507
weighted avg       0.87      0.87      0.87       507

